In [1]:
import pandas as pd
import numpy as np
from replay.metrics import *
from replay.metrics import Median, ConfidenceInterval, PerUser


libgomp: Invalid value for environment variable OMP_NUM_THREADS


### Создание датасета

In [2]:
train_interaction = {  
    'Anton' : {'All along the watchtower' : 1, 'Kalinka-Malinka':1, 'Nothing else metter' : 1, 'Smells Like Teen Spirit' : 1},
    'Lesha' : {'Kalinka-Malinka':1, 'Ty nesi menya reka' :1,'Nothing else metter':1, 'Techet Reka Volga' : 1 },
    'Anna' : {'Nothing else metter' : 1, 'Tam gde klen shumit' :1, 'All along the watchtower' : 1},
    'Dasha' : {'All along the watchtower' : 1, 'Look at me now' : 1, 'Smells Like Teen Spirit' : 1, 'All along the watchtower' : 1},
    'Sasha' : {'Kalinka-Malinka':1, 'Techet Reka Volga' : 1 , 'Nothing else metter':1,  'Tam gde klen shumit' :1}}


In [3]:
test_interaction = {'query_id': ['Anton', 'Lesha', 'Anna', 'Dasha', 'Sasha'],
 'item_id': ['All along the watchtower', 'Tam gde klen shumit', 'Nothing else metter', 'Nothing else metter', 'Ty nesi menya reka'],
 'ratings' : [1, 1, 1, 1, 1]}

In [4]:
train = pd.DataFrame(columns = ['query_id', 'item_id', 'rating'])
index = 0
for name in train_interaction.keys():
    for value in train_interaction[name]:
        train.loc[index] = [name, value, '1']
        index += 1

train

,query_id,item_id,rating
0,Anton,All along the watchtower,1
1,Anton,Kalinka-Malinka,1
2,Anton,Nothing else metter,1
3,Anton,Smells Like Teen Spirit,1
4,Lesha,Kalinka-Malinka,1
5,Lesha,Ty nesi menya reka,1
6,Lesha,Nothing else metter,1
7,Lesha,Techet Reka Volga,1
8,Anna,Nothing else metter,1
9,Anna,Tam gde klen shumit,1


In [5]:
test = pd.DataFrame(test_interaction)
test

,query_id,item_id,ratings
0,Anton,All along the watchtower,1
1,Lesha,Tam gde klen shumit,1
2,Anna,Nothing else metter,1
3,Dasha,Nothing else metter,1
4,Sasha,Ty nesi menya reka,1


### Метрики

In [6]:
def hit_rate(test:pd.DataFrame, predict:pd.DataFrame, k:int) -> float:
    hit_rate_list = []
    user_pred = set(predict['query_id'].unique())
    user_test = set(test['query_id'].unique())
    user_union = set.intersection(user_pred, user_test)
    for user in user_union:
        test_seq = test[test['query_id'] == user]['item_id'].values
        pred_seq = pred[pred['query_id'] == user]['item_id'].values
        if pred_seq[:k] in test_seq:
            hit_rate_list.append(1)
        else:
            hit_rate_list.append(0)
            
    no_test_users = user_test - user_union
    hit_rate_list.extend([0] * len(no_test_users))

    return np.mean(hit_rate_list)


def coverage(train:pd.DataFrame, predict:pd.DataFrame, k:int) -> float:
    unique_item = set(train['item_id'])
    predict_topk = predict.groupby('query_id').agg(lambda x: x[:k]).explode('item_id')['item_id']
    unique_predict = set(predict_topk)
    return len(unique_predict & unique_item) / len(unique_item)


def precision(test:pd.DataFrame, predict:pd.DataFrame, k:int) -> float:
    res = []
    user_pred = set(predict['query_id'].unique())
    user_test = set(test['query_id'].unique())
    user_union = set.intersection(user_pred, user_test)
    for user in user_union:
        test_seq = test[test['query_id'] == user]['item_id'].values
        pred_seq = pred[pred['query_id'] == user]['item_id'].values
        if pred_seq in test_seq:
            ans = len(set(pred_seq[:k]).intersection(set(test_seq[:k])))
            res.append(ans / k)
        else:
            res.append(0)
            
    no_test_users = user_test - user_union
    res.extend([0] * len(no_test_users))

    return np.mean(res)


def recall(test: pd.DataFrame, predict : pd.DataFrame, k:int) -> float:
    res = []
    user_pred = set(predict['query_id'].unique())
    user_test = set(test['query_id'].unique())
    user_union = set.intersection(user_pred, user_test)
    for user in user_union:
        test_seq = test[test['query_id'] == user]['item_id'].values
        pred_seq = pred[pred['query_id'] == user]['item_id'].values
        if pred_seq in test_seq:
            ans = len(set(pred_seq[:k]).intersection(set(test_seq[:k])))
            res.append(ans / len(set(test_seq)))
        else:
            res.append(0)
            
    no_test_users = user_test - user_union
    res.extend([0] * len(no_test_users))

    return np.mean(res)

#### Порекомендуем всем самую прослушиваемую песню

In [7]:
train.groupby('item_id').agg('count')['rating'].sort_values(ascending = False)

item_id
Nothing else metter         4
All along the watchtower    3
Kalinka-Malinka             3
Smells Like Teen Spirit     2
Tam gde klen shumit         2
Techet Reka Volga           2
Look at me now              1
Ty nesi menya reka          1
Name: rating, dtype: int64

In [8]:
pred = {'query_id': ['Anton', 'Lesha', 'Anna', 'Dasha', 'Sasha'],
 'item_id': ['Nothing else metter', 'Nothing else metter', 'Nothing else metter', 'Nothing else metter', 'Nothing else metter'],
  'rating' : [1, 1, 1, 1, 1]}
pred = pd.DataFrame(pred)
pred

,query_id,item_id,rating
0,Anton,Nothing else metter,1
1,Lesha,Nothing else metter,1
2,Anna,Nothing else metter,1
3,Dasha,Nothing else metter,1
4,Sasha,Nothing else metter,1


In [9]:
test

,query_id,item_id,ratings
0,Anton,All along the watchtower,1
1,Lesha,Tam gde klen shumit,1
2,Anna,Nothing else metter,1
3,Dasha,Nothing else metter,1
4,Sasha,Ty nesi menya reka,1


In [10]:
print('Hit Rate = {}'.format(hit_rate(test, pred, 1)))
print('Coverage = {}'.format(coverage(train, pred, 1))) 
print('Precision = {}'.format(precision(test, pred, 1)))
print('Recall = {}'.format(recall(test, pred, 1)))

Hit Rate = 0.4
Coverage = 0.125
Precision = 0.4
Recall = 0.4


### Метрики посчитанные в RePlay

In [11]:
print(HitRate(1)(pred, test))
print(Coverage(1)(pred, train))
print(Precision(1)(pred, test))
print(Recall(1)(pred, test))

{'HitRate@1': 0.4}
{'Coverage@1': 0.125}
{'Precision@1': 0.4}
{'Recall@1': 0.4}


In [12]:
metrics = [
     Precision(1),
     Precision(1, mode=PerUser()),
     Precision(1, mode=Median()),
     Precision(1, mode=ConfidenceInterval(alpha=0.95)),
     Recall(1),
     MAP(1),
     MRR(1),
     NDCG(1),
     HitRate(1),
     RocAuc(1),
     Coverage(1),
     Novelty(1),
     Surprisal(1),
 ]

OfflineMetrics(metrics)(pred, test, train)

{'Precision@1': 0.4,
 'Precision-PerUser@1': {'Anna': 1.0,
  'Anton': 0.0,
  'Dasha': 1.0,
  'Lesha': 0.0,
  'Sasha': 0.0},
 'Precision-Median@1': 0.0,
 'Precision-ConfidenceInterval@1': 0.480091167635531,
 'Recall@1': 0.4,
 'MAP@1': 0.4,
 'MRR@1': 0.4,
 'NDCG@1': 0.4,
 'HitRate@1': 0.4,
 'RocAuc@1': 0.4,
 'Coverage@1': 0.125,
 'Novelty@1': 0.2,
 'Surprisal@1': 0.13864688385321391}

### Проверяем метрики на прочность =)

#### Рекомендуем для @2

In [13]:
assert coverage(train, pred, 2) == Coverage(2)(pred, train)['Coverage@2']
assert hit_rate(test, pred, 2) == HitRate(2)(pred, test)['HitRate@2']
assert precision(test, pred, 2) == Precision(2)(pred, test)['Precision@2']
assert recall(test, pred, 2) == Recall(2)(pred, test)['Recall@2']

#### Рекомендуем не для всех пользователей

In [14]:
pred = {'query_id': ['Anton', 'Lesha', 'Anna', 'Sasha'],
 'item_id': ['Nothing else metter', 'Nothing else metter', 'Nothing else metter', 'Nothing else metter'],
  'rating' : [1, 1, 1, 1]}
pred = pd.DataFrame(pred)
pred

,query_id,item_id,rating
0,Anton,Nothing else metter,1
1,Lesha,Nothing else metter,1
2,Anna,Nothing else metter,1
3,Sasha,Nothing else metter,1


In [15]:
assert coverage(train, pred, 1) == Coverage(1)(pred, train)['Coverage@1']
assert hit_rate(test, pred, 1) == HitRate(1)(pred, test)['HitRate@1']
assert precision(test, pred, 1) == Precision(1)(pred, test)['Precision@1']
assert recall(test, pred, 1) == Recall(1)(pred, test)['Recall@1']

#### Делаем рекомендацию для того, кого нет в test

In [16]:
pred = {'query_id': ['Anton','Ksusha', 'Lesha', 'Anna', 'Sasha'],
 'item_id': ['Nothing else metter', 'Nothing else metter', 'Nothing else metter', 'Nothing else metter', 'Nothing else metter'],
  'rating' : [1, 1, 1, 1, 1]}
pred = pd.DataFrame(pred)
pred

,query_id,item_id,rating
0,Anton,Nothing else metter,1
1,Ksusha,Nothing else metter,1
2,Lesha,Nothing else metter,1
3,Anna,Nothing else metter,1
4,Sasha,Nothing else metter,1


In [17]:
test

,query_id,item_id,ratings
0,Anton,All along the watchtower,1
1,Lesha,Tam gde klen shumit,1
2,Anna,Nothing else metter,1
3,Dasha,Nothing else metter,1
4,Sasha,Ty nesi menya reka,1


In [18]:
assert coverage(train, pred, 1) == Coverage(1)(pred, train)['Coverage@1']
assert hit_rate(test, pred, 1) == HitRate(1)(pred, test)['HitRate@1']
assert precision(test, pred, 1) == Precision(1)(pred, test)['Precision@1']
assert recall(test, pred, 1) == Recall(1)(pred, test)['Recall@1']